In [ ]:
# Basic
from collections import Counter
import math
import numpy as np
import os
import pprint
import sys

# yaml specific
import yaml

# Data handling
import pandas as pd
from tqdm import tqdm

# Appending root folder to path for easy importing
sys.path.append("../")

# my library
from src.db_utils import update_player, add_player 

In [ ]:
# Config variables
raw_data_path = os.path.join("..", "raw_data") 
clean_data_path = os.path.join("..", "clean_data") 

### Utility functions. 

In [ ]:
def get_clean_teams(major_teams):
    # remove bad chars from string
    bad_chars = [';', ':', '!', "*", "'", '"', '[', ']', ',']
    teams = []
    for team in major_teams.split(", "):
        for char in bad_chars:
            team = team.replace(char, "")
        teams.append(team.strip().rstrip())
    return teams

# This notebook will be used to curate 4 tables. These tables are independent of matches
## 1) Country 2) Tournament 3) Team 4) Player
### Initially handling everything through CSVs. Later, we will see if using a DB is required

In [ ]:
players = pd.read_csv(os.path.join(raw_data_path, "players.csv"))
players = players.loc[:, ~players.columns.str.contains('^Unnamed')]

## Country table

### Focusing only on the top 20 countries with professional cricket players

In [ ]:
countries = []
for key, value in players["COUNTRY"].value_counts().nlargest(20).iteritems():
    countries.append(key)

In [ ]:
country_columns = ["country_id", "country_name"]
country_csv = pd.DataFrame(columns = country_columns)
for i in range(len(countries)):
    country_id = i+1
    country_name = countries[i]
    country_csv = country_csv.append({"country_id" : country_id, 
                                      "country_name" : country_name}, ignore_index=True)
country_csv.to_csv(os.path.join(clean_data_path, "country.csv"), index=False)

## Tournament table

### Each zip file in https://cricsheet.org/ is going to be considered as a separate tournament. This list is going to be manually curated due to its short size. These are the only matches we will be caring about

In [ ]:
tournaments = [
    {
        "tournament_name" : "TEST",
        "tournament_format" : "TEST",
        "tournament_full_name" : "Test International Match"
    },
    {
        "tournament_name" : "ODI",
        "tournament_format" : "ODI",
        "tournament_full_name" : "ODI Match"
    },
    {
        "tournament_name" : "T20I",
        "tournament_format" : "T20",
        "tournament_full_name" : "T20 International Match"
    },
    {
        "tournament_name" : "BBL",
        "tournament_format" : "T20",
        "tournament_full_name" : "Big Bash League"
    },
    {
        "tournament_name" : "IPL",
        "tournament_format" : "T20",
        "tournament_full_name" : "Indian Premier Leargue"
    },
    {
        "tournament_name" : "CPL",
        "tournament_format" : "T20",
        "tournament_full_name" : "Caribbean Premier Leargue"
    },
    {
        "tournament_name" : "T20B",
        "tournament_format" : "T20",
        "tournament_full_name" : "T20 Blast"
    },
    {
        "tournament_name" : "PSL",
        "tournament_format" : "T20",
        "tournament_full_name" : "Pakistan Super League"
    },
    {
        "tournament_name" : "SSM",
        "tournament_format" : "T20",
        "tournament_full_name" : "Super Smash"
    },
    {
        "tournament_name" : "WBB",
        "tournament_format" : "T20",
        "tournament_full_name" : "Womens Big Bash League"
    }
]

In [ ]:
tournament_columns = ["tournament_id", "tournament_name", "tournament_format", "tournament_full_name"]
tournament_csv = pd.DataFrame(columns = tournament_columns)
for i in range(len(tournaments)):
    tournament_id = i+1
    tournament_name = tournaments[i]["tournament_name"]
    tournament_format = tournaments[i]["tournament_format"]
    tournament_full_name = tournaments[i]["tournament_full_name"]
    tournament_csv = tournament_csv.append({"tournament_id" : tournament_id, 
                                      "tournament_name" : tournament_name,
                                      "tournament_format" : tournament_format, 
                                      "tournament_full_name" : tournament_full_name}, ignore_index=True)
tournament_csv.to_csv(os.path.join(clean_data_path, "tournament.csv"), index=False)

## Team table


### Considering only those teams where the players belong to the above 20 countries and aged < 50 (as of 2019)

In [ ]:
# considering only the top 10 countries as the remaining countries barely have 10 important players that can be seperately ingested. 
required_countries = countries[:-10]
required_columns = ['NAME', 'COUNTRY', 'Full name', 'Birthdate', 'Major teams', 'Batting style', 'Bowling style', 'Other']
required_players = players[(players["COUNTRY"].isin(countries)) & (players["Age"] <= 55)][required_columns]

In [ ]:
teams = []
for index, row in tqdm(required_players.iterrows()):
    # skip columns that are not string
    if type(row["Major teams"]) is not str:
        continue
    for team in get_clean_teams(row["Major teams"]):
        teams.append(team)

### If a team has less than 'threshold' registered professional players, ignore that team

In [ ]:
threshold = 25
teams_set = set()
for team, count in Counter(teams).most_common():
    if count > threshold:
        teams_set.add(team)
teams_list = list(teams_set)

In [ ]:
len(teams_list)

In [ ]:
team_csv = {}
for i in tqdm(range(len(teams_list))):
    team_id = i+1
    team_name = teams_list[i]
    team_label = "".join(word[0] for word in team_name.split(" "))
    team_csv[team_name] = {
        "team_id" : team_id, 
        "team_name" : team_name, 
        "team_label" : team_label
    }
team_csv["Delhi Capitals"] = {
    "team_id" : team_csv["Delhi Daredevils"]["team_id"], 
    "team_name" : "Delhi Capitals", 
    "team_label" : "DC"
}

team_csv["Punjab Kings"] = {
    "team_id" : team_csv["Kings XI Punjab"]["team_id"], 
    "team_name" : "Punjab Kings", 
    "team_label" : "PK"
}

df_team_csv = pd.DataFrame.from_dict(team_csv, "index")
df_team_csv.to_csv(os.path.join(clean_data_path, "team.csv"), index=False)

## Player table

### Considering only those players who belong to the 20 countries and aged < 50 (as of 2019)

### Utility maps to ingest players

In [ ]:
df_country = pd.read_csv(os.path.join(clean_data_path, "country.csv"))
df_country = df_country.loc[:, ~df_country.columns.str.contains('^Unnamed')]
country_id_map = dict(zip(df_country.country_name, df_country.country_id))

In [ ]:
df_team = pd.read_csv(os.path.join(clean_data_path, "team.csv"))
df_team = df_team.loc[:, ~df_team.columns.str.contains('^Unnamed')]
team_id_map = dict(zip(df_team.team_name, df_team.team_id))

In [ ]:
player_columns = ["player_id", "player_display_name", "player_name", "player_full_name", "batting_style", "bowling_style", "birthdate", "country_id"]
player_csv = {}
i = 0 # player tracker
team_threshold = 2 # Keep only those players who play in atleast these many teams
for index, row in tqdm(required_players.iterrows()):
    
    # If player doesnt have a proper team skip
    if type(row["Major teams"]) is not str:
        continue
    
    # Getting the list of team ids this player has played in
    team_list_ids = ""
    for team in get_clean_teams(row["Major teams"]):
        if team in team_id_map:
            team_list_ids += str(team_id_map[team]) + ","
            
    # If the player doesnt play in atleast 2 teams skip them
    if len(team_list_ids.split(",")) < team_threshold+1 :
        continue
        
    # removing last comma
    team_list_ids = team_list_ids[:-1]
    
    player_id = i+1
    player_display_name = "" # This needs to be updated later with fuzzy matching
    player_name = row["NAME"]
    player_full_name = row["Full name"]
    batting_style = row["Batting style"]
    bowling_style = row["Bowling style"]
    birthdate = row["Birthdate"]
    country_id = country_id_map[row["COUNTRY"]]
    list_team_ids = team_list_ids
    
    player_csv[i] = {
                        "player_id" : player_id, 
                        "player_display_name" : player_display_name,
                        "player_name" : player_name,
                        "player_full_name" : player_full_name,
                        "batting_style" : batting_style,
                        "bowling_style" : bowling_style,
                        "birthdate" : birthdate,
                        "country_id" : country_id,
                        "list_team_ids" : list_team_ids
    }
    
    i += 1
    
df_player_csv = pd.DataFrame.from_dict(player_csv, "index")
df_player_csv.to_csv(os.path.join(clean_data_path, "player.csv"), index=False)

In [ ]:
len(player_csv)

### Use this cell for all manual corrections to update/insert in players table

In [ ]:

# inserts

# IPL
add_player("Ankit Soni", "Ankit Soni", "Ankit Soni", "Right-hand bat", "Legbreak googly", "02/08/1993", "2", str(team_id_map["Gujarat Lions"]))
add_player("J Suchith", "Jagadeesha Suchith", "Jagadeesha Suchith", "Left-hand bat", "Slow left-arm orthodox", "01/16/1994", "2", str(team_id_map["Kings XI Punjab"]))
add_player("SD Lad", "Siddhesh Lad", "Siddhesh Dinesh Lad", "Right-hand bat", "Right-arm offbreak", "23/05/92", "2", str(team_id_map["Mumbai Indians"]))
add_player("S Kaushik", "Shivil Kaushik", "Shivil Sharma Kaushik", "Left-hand bat", "Slow left-arm wrist-spin", "07/09/95", "2", str(team_id_map["Gujarat Lions"]))
add_player("KM Asif", "KM Asif", "KM Asif", "Right-hand bat", "Right-arm medium", "24/07/93", "2", str(team_id_map["Chennai Super Kings"]))
add_player("AS Roy", "Anukul Roy", "Anukul Sudhakar Roy", "Left-hand bat", "Slow left-arm orthodox", "30/11/98", "2", str(team_id_map["Mumbai Indians"]))
add_player("YBK Jaiswal", "Yashasvi Jaiswal", "Yashasvi Bhupendra Kumar Jaiswal", "Left-hand bat", "", "28/12/01", "2", str(team_id_map["Rajasthan Royals"]))
add_player("Abdul Samad", "Abdul Samad", "Abdul Samad", "Right-hand bat", "Right-arm legbreak", "10/28/01", "2", str(team_id_map["Sunrisers Hyderabad"]))
add_player("Kartik Tyagi", "Kartik Tyagi", "Kartik Tyagi", "Right-hand bat", "Right-arm fast", "11/08/00", "2", str(team_id_map["Rajasthan Royals"]))
add_player("Lalit Yadav", "Lalit Yadav", "Lalit Yadav", "Right-hand bat", "Right-arm offbreak", "03/01/97", "2", str(team_id_map["Delhi Daredevils"]))
add_player("Ravi Bishnoi", "Ravi Bishnoi", "Ravi Bishnoi", "Right-hand bat", "Legbreak googly", "05/09/00", "2", str(team_id_map["Kings XI Punjab"]))
add_player("RA Shaikh", "Rahil Shaikh", "Rahil Akhil Ahmed Shaikh", "Left-hand bat", "Left-arm medium", "12/06/85", "2", str(team_id_map["Mumbai Indians"]))
add_player("AN Ahmed", "AN Ahmed", "AN Ahmed", "Left-hand bat", "Left-arm medium", "12/06/85", "2", str(team_id_map["Mumbai Indians"]))
add_player("AA Kazi", "Abrar Kazi", "Abrar Anjum Kazi", "Left-hand bat", "Slow left-arm orthodox", "10/29/89", "2", str(team_id_map["Royal Challengers Bangalore"]))
add_player("T Mishra", "Tanmay Mishra", "Tanmay Mishra", "Right-hand bat", "Right-arm medium-fast", "12/22/86", "2", str(team_id_map["Deccan Chargers"]))
# 2021
add_player("", "Bhagath Varma", "Kanumuri Bhagath Varma", "Right-hand bat", "Right-arm offbreak", "", "2", "50")
add_player("", "C Hari Nishaanth", "Chezhian Nishaanth", "Left-hand bat", "Right-arm offbreak", "", "2", "50")
add_player("", "Ravisrinivasan Sai Kishore", "Ravisrinivasan Sai Kishore", "Left-hand bat", "Slow left-arm orthodox", "", "2", "50")
add_player("", "Lukman Meriwala", "Lukman Iqbal Meriwala", "Left-hand bat", "Left-arm fast-medium", "", "2", "322")
add_player("", "Ripal Patel", "Ripal Vinubhai Patel", "Right-hand bat", "Right-arm medium-fast", "", "2", "322")
add_player("", "Manimaran Siddharth", "Manimaran Siddharth", "Right-hand bat", "Slow left-arm orthodox", "", "2", "322")
add_player("", "Vaibhav Arora", "Vaibhav Gopal Arora", "Right-hand bat", "Right-arm fast-medium", "", "2", "356")
add_player("", "Marco Jansen", "Marco Jansen", "Right-hand bat", "Left-arm fast", "", "3", "86")
add_player("", "Yudhvir Singh", "Yudhvir Singh Charak", "Right-hand bat", "Right-arm medium-fast", "", "2", "86")
add_player("", "Prabhsimran Singh", "Prabhsimran Singh", "Right-hand bat", "", "", "2", "591")
add_player("", "Utkarsh Singh", "Utkarsh Rajiv Singh", "Left-hand bat", "Right-arm offbreak", "", "2", "591")
add_player("", "Akash Singh", "Akash Singh", "Right-hand bat", "", "", "2", "113")
add_player("", "Kuldip Yadav", "Kuldip Yadav", "", "", "", "2", "113")
add_player("", "Finn Allen", "Finnley Hugh Allen", "Right-hand bat", "", "", "7", "567")
add_player("", "Pavan Deshpande", "Pavan Uday Deshpande", "Left-hand bat", "Right-arm offbreak", "", "2", "567")
add_player("", "Mohammed Azharuddeen", "Mohammed Azharuddeen", "Right-hand bat", "", "", "2", "567")
add_player("", "Suyash Prabhudessai", "Suyash S Prabhudessai", "Right-hand bat", "Right-arm medium", "", "2", "567")
add_player("", "Shahbaz Ahmed", "Shahbaz Ahmed", "Left-hand bat", "Slow left-arm orthodox", "", "2", "567")
add_player("", "Virat Singh", "Virat Binod Singh", "Left-hand bat", "Legbreak googly", "", "2", "136")
add_player("RV Patel", "Ripal Patel", "Ripal Patel", "Right-hand bat", "Legbreak googly", "Right-arm medium-fast", "2", "136")

# BBL
add_player("AG Harriott", "Andrew Harriott", "Andrew Harriott", "Right-hand bat", "Right-arm medium-fast", "05/03/92", "4", str(team_id_map["Melbourne Renegades"]))
add_player("B Doggett", "Brendan Doggett", "Brendan Doggett", "Right-hand bat", "Right-arm fast-medium", "03/05/94", "4", str(team_id_map["Sydney Thunder"]) + "," + str(team_id_map["Brisbane Heat"]))
add_player("CP Simpson", "Chris Simpson", "Christopher Patrick Simpson", "Right-hand bat", "Right-arm offbreak", "09/01/82", "4", str(team_id_map["Sydney Thunder"]))
add_player("D Morton", "Daniel Morton", "Daniel Morton", "Right-hand bat", "Right-arm offbreak", "", "4", str(team_id_map["Perth Scorchers"]))
add_player("Dilbar Hussain", "Dilbar Hussain", "Dilbar Hussain", "Right-hand bat", "Right-arm fast-medium", "", "5", str(team_id_map["Melbourne Stars"]))
add_player("GA West", "GA West", "GA West", "", "", "", "", str(team_id_map["Adelaide Strikers"]))
add_player("H Kerr", "Hayden Kerr", "Hayden Kerr", "", "", "", "", str(team_id_map["Sydney Sixers"]))
add_player("Haris Rauf", "Haris Rauf", "Haris Rauf", "Right-hand bat", "Right-arm fast", "11/07/93", "5", str(team_id_map["Melbourne Stars"]))
add_player("J Fraser-McGurk", "Jake Fraser-McGurk", "Jake Fraser-McGurk", "Right-hand bat", "Legbreak googly", "11/04/02", "4", str(team_id_map["Melbourne Renegades"]))
add_player("CR Swan", "CR Swan", "CR Swan", "", "", "", "4", "72")
add_player("JA Prestwidge", "JA Prestwidge", "JA Prestwidge", "", "", "", "4", "199, 72")
add_player("JP Wood", "JP Wood", "JP Wood", "", "", "", "4", "72")
add_player("K White (sub)", "K White (sub)", "K White (sub)", "", "", "", "4", "512")
add_player("L Bowe", "L Bowe", "L Bowe", "", "", "", "4", "512, 568")
add_player("LR Morris", "LR Morris", "LR Morris", "", "", "", "4", "568")
add_player("M Perry", "M Perry", "M Perry", "", "", "", "4", "199")
add_player("MJ Owen", "MJ Owen", "MJ Owen", "", "", "", "4", "45")
add_player("NA McSweeney", "NA McSweeney", "NA McSweeney", "", "", "", "4", "199")
add_player("NT Ellis", "NT Ellis", "NT Ellis", "", "", "", "4", "45")
add_player("Noor Ahmad", "Noor Ahmad", "Noor Ahmad", "", "", "", "4", "199")
add_player("O Davies", "O Davies", "O Davies", "", "", "", "4", "512")
add_player("P Hatzoglou", "P Hatzoglou", "P Hatzoglou", "", "", "", "4", "199")
add_player("RJG Lockyear", "RJG Lockyear", "RJG Lockyear", "", "", "", "4", "45, 512")
add_player("RR Ayre (sub)", "RR Ayre (sub)", "RR Ayre (sub)", "", "", "", "4", "455")
add_player("SJ Coyte (2)", "SJ Coyte (2)", "SJ Coyte (2)", "", "", "", "4", "512")
add_player("T Sangha", "T Sangha", "T Sangha", "", "", "", "4", "512")
add_player("XC Bartlett", "XC Bartlett", "XC Bartlett", "", "", "", "4", "72")
add_player("K White", "K White", "K White", "", "", "", "4", "447")
add_player("RR Ayre", "RR Ayre", "RR Ayre", "", "", "", "4", "241")

